In [1]:
from urllib import request
from bs4 import BeautifulSoup
from pandas import *

In [2]:
list_tournois = {  
    "2016" : "http://www.lequipe.fr/Tennis/SH_RG.html" ,
    "2015" : "http://www.lequipe.fr/Tennis/TennisTableauTournoi5637.html",
    "2014" : "http://www.lequipe.fr/Tennis/TennisTableauTournoi4939.html",
    "2013" : "http://www.lequipe.fr/Tennis/TennisTableauTournoi4456.html",
    "2012" : "http://www.lequipe.fr/Tennis/TennisTableauTournoi3687.html",
    "2011" : "http://www.lequipe.fr/Tennis/TennisTableauTournoi3114.html",
    "2010" :"http://www.lequipe.fr/Tennis/TennisTableauTournoi2672.html"
 }

In [3]:
def is_joueur(tag):
    return tag.has_attr('nomcomplet')

In [7]:
winners = []
losers = []
years = []
scores_winners = []
scores_losers = []

for year in list_tournois.keys():
    data = request.urlopen(list_tournois[year]).read()
    soup = BeautifulSoup(data, 'html.parser')
    
    table = soup.find(id = "rateau")

    list_rounds = []
    for i in [4,5,6]: #I only take the three last rounds. It is possible to take more ! 
        list_rounds+=table.findAll("div", attrs = {"numtour": str(i)})

    list_match = []
    for i in list_rounds:
            list_match += i.findAll("div", attrs = {"class": "joueurs"}) 

    for v,i in enumerate(list_match):
        years+=[year]
        for j in i.findAll("a"):
            if "gagne" in j["class"]:
                winners += [j.findAll(is_joueur)[0]["nomcomplet"] ]
                scores_winners += [j.findAll(attrs={"class": "score"})[0].text]
            else:
                losers += [j.findAll(is_joueur)[0]["nomcomplet"] ]
                scores_losers += [j.findAll(attrs={"class": "score"})[0].text]


In [8]:
df = DataFrame({"Winners": winners, "Losers": losers, "Years" : years,
                "Scores_Winners": scores_winners, "Scores_Losers": scores_losers})
df.head()

,Losers,Scores_Losers,Scores_Winners,Winners,Years
0,TSONGA,177661,65576,DJOKOVIC,2012
1,DEL POTRO,674203,36666,FEDERER,2012
2,MURRAY,47332,6666,FERRER,2012
3,ALMAGRO,6423,766,NADAL,2012
4,FEDERER,453,676,DJOKOVIC,2012


In [9]:
df.to_csv("rolland_garros.csv", index=False)